1. 구글 지도 검색 키워드
2. data
    store_point 평점 평균
    review_text 더보기에서 여러개들 스크랩
3. 검색이 안될 경우 패스
    리뷰가 없을 경우도 패스


In [90]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import ijson
import numpy as np
import time

#options = Options()
#options.binary_location = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe'
chromedriver='C:/Users/default.DESKTOP-FVT3076/Desktop/crawl/chromedriver.exe'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chromedriver = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe'
driver = webdriver.Chrome(chromedriver,options=chrome_options)
time.sleep(1)

In [ ]:
# 파일 읽기
# f = open(f"./data/data.json", encoding="UTF-8")
# objects = ijson.items(f, 'item')
# df = pd.DataFrame(objects)

df = pd.read_csv('../data/google map용/GoogleMap용_가게정보_part_5.csv', sep=',', encoding='utf-8')
start = 32438
end = start + 20000

df = df[start:end]
print(df)

In [54]:
df.rename(columns={"naver_keyword":"google_keyword"},inplace=True)

In [ ]:
review_stars_list = [] # 개별 평점
review_time_list = [] # 개별 리뷰 작성 시간
review_list = []
star_avg_list = []

for i, keyword in enumerate(tqdm(df['google_keyword'])):
    
    google_map_search_url = f"https://www.google.com/maps/search/{keyword}"
    driver.get(google_map_search_url)
    time.sleep(1)
    try:
        # 별점 
        star_review_stars = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span").text
        
        more_reviews = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span[1]/span[2]/span[1]/button")
        more_reviews.send_keys(Keys.ENTER)
        time.sleep(2)
       
        # 리뷰개수만큼 스크롤
        #Find the total number of reviews
        total_number_of_reviews = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[2]').text.split(" ")[1][:-1]
        total_number_of_reviews = int(total_number_of_reviews.replace(',','')) if ',' in total_number_of_reviews else int(total_number_of_reviews)
        #Find scroll layout
        scrollable_div = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')
        #Scroll as many times as necessary to load all reviews
        for i in range(0,(round(total_number_of_reviews/10 - 1))):
                driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
                        scrollable_div)
                time.sleep(1)

        review_text_list = [] # 임시 선언

        response = BeautifulSoup(driver.page_source, 'html.parser')
        result_set = response.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')
        rev_dict = {'Review Rate': [],
            'Review Time': [],
            'Review Text' : []}
        
        for result in result_set:
            review_rate = result.find('span', class_='ODSEW-ShBeI-H1e3jb')["aria-label"]
            review_time = result.find('span',class_='ODSEW-ShBeI-RgZmSc-date').text
            review = result.find('span',class_='ODSEW-ShBeI-text').text
            rev_dict['Review Rate'].append(review_rate)
            rev_dict['Review Time'].append(review_time)
            rev_dict['Review Text'].append(review)
            review_text_list.append(review)
        res = pd.DataFrame(rev_dict)
#         print(res)
        review_stars_list.append(rev_dict['Review Rate'])
        review_time_list.append(rev_dict['Review Time'])
#         review_text = ','.join(review_text_list)
        review_list.append(rev_dict['Review Text']) # 일단 join 하지말고 list로
        star_avg_list.append(star_review_stars)

    # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
    except Exception as e1:
        print(f"{i}행 문제가 발생")
        print(e1)
        # 리뷰가 없으므로 null을 임시로 넣어줍니다.
        review_stars_list.append('null')
        review_time_list.append('null')
        review_list.append('null')
        star_avg_list.append('null')
        
# driver.quit()
df['google_review_date'] = review_time_list # 리뷰 작성 시간
df['google_stars'] = review_stars_list
df['google_star_avg'] = star_avg_list  # 상세페이지에서 평가한 별점 평균
df['google_review_txt'] = review_list  # 상세페이지에 나온 리뷰 텍스트들

1행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

2행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

4행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

5행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome

57행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

58행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

59행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

62행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless ch

103행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

104행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

106행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

108행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headles

161행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

163행 문제가 발생
Message: element not interactable
  (Session info: headless chrome=94.0.4606.61)

164행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

167행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

168행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/d

207행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

208행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

209행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

210행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headles

...

11181행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11182행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11183행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11185행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info:

11242행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11243행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11245행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11246행 문제가 발생
Message: element not interactable
  (Session info: headless chrome=94.0.4606.61)

11247행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/di

11320행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11322행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11324행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11326행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info:

11385행 문제가 발생
Message: element not interactable
  (Session info: headless chrome=94.0.4606.61)

11386행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11388행 문제가 발생
Message: element not interactable
  (Session info: headless chrome=94.0.4606.61)

11389행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11390행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11391행 문제가 발생
Message: no such ele

11434행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11436행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11437행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info: headless chrome=94.0.4606.61)

11439행 문제가 발생
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span"}
  (Session info:

In [ ]:
# 도중에 중단 되면 된거라도 저장
length = len(review_time_list)
temp = df[:length]
temp['google_review_date'] = review_time_list # 리뷰 작성 시간
temp['google_stars'] = review_stars_list
temp['google_star_avg'] = star_avg_list  # 상세페이지에서 평가한 별점 평균
temp['google_review_txt'] = review_list  # 상세페이지에 나온 리뷰 텍스트들
df = temp.loc[temp['google_review_txt'] != 'null']

In [ ]:
# 성공적으로 된 후 NaN 데이터 삭제
df = df.loc[df['google_star_avg'] != 'null']
df

In [63]:
# ElasticSearch용
import os
from datetime import datetime

temp = df[['name','area', 'address','latitude','longitude']]
today = str(datetime.date(datetime.today()))
folder = '../outputs/ES용 data/'
file = 'ES_' + str(start) + '~' + str(end) + '행_' + today + '.csv'

if os.path.isfile(file):
    os.remove(file)
temp.to_csv(folder + file, encoding='utf-8')
# df.to_csv(today + '_test.csv', encoding='cp949')
print(file)

ES_0~18000행_2021-10-01.csv


In [64]:
# Bert용

temp = df[['google_keyword', 'google_review_date','google_stars','google_star_avg','google_review_txt']]
today = str(datetime.date(datetime.today()))
folder = '../outputs/BERT용 data/'
file = 'BERT_' + str(start) + '~' + str(end) + '행_' + today + '.csv'

if os.path.isfile(file):
    os.remove(file)
temp.to_csv(folder + file, encoding='utf-8')
print(file)

BERT_0~18000행_2021-10-01.csv
